In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
from tqdm.notebook import tqdm
from pandas_profiling import ProfileReport
from itertools import islice
from datetime import datetime
import pprint
import ast
from catboost.text_processing import Tokenizer
import pprint
import ast
from catboost.text_processing import Tokenizer
from collections import Counter


pp = pprint.PrettyPrinter(indent=4)


def take(n, iterable):
    """Return first n items of the iterable as a list"""
    return list(islice(iterable, n))


food_data_train = pd.read_csv("./input/main_task.csv")
food_data_test = pd.read_csv("./input/kaggle_task.csv")

world_cityes = pd.read_csv("./input/worldcities.csv")
country_population = pd.read_csv("./input/pop_by_country.csv")


food_data_train["sample"] = 1  # it's traning dataset
food_data_test["sample"] = 0  # it's test dataset
food_data_test["Rating"] = 0  # equal 0 for prediction

food_data = food_data_test.append(food_data_train, sort=False).reset_index(
    drop=True
)


pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)
display(food_data.head(5))
display(country_population.head(5))

,Restaurant_id,City,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,sample,Rating
0,id_0,Paris,"['Bar', 'Pub']",12963.0,$$ - $$$,4.0,"[[], []]",/Restaurant_Review-g187147-d10746918-Reviews-L...,d10746918,0,0.0
1,id_1,Helsinki,"['European', 'Scandinavian', 'Gluten Free Opti...",106.0,$$ - $$$,97.0,"[['Very good reviews!', 'Fine dining in Hakani...",/Restaurant_Review-g189934-d6674944-Reviews-Ra...,d6674944,0,0.0
2,id_2,Edinburgh,['Vegetarian Friendly'],810.0,$$ - $$$,28.0,"[['Better than the Links', 'Ivy Black'], ['12/...",/Restaurant_Review-g186525-d13129638-Reviews-B...,d13129638,0,0.0
3,id_3,London,"['Italian', 'Mediterranean', 'European', 'Vege...",1669.0,$$$$,202.0,"[['Most exquisite', 'Delicious and authentic']...",/Restaurant_Review-g186338-d680417-Reviews-Qui...,d680417,0,0.0
4,id_4,Bratislava,"['Italian', 'Mediterranean', 'European', 'Seaf...",37.0,$$$$,162.0,"[['Always the best in bratislava', 'Very good ...",/Restaurant_Review-g274924-d1112354-Reviews-Ma...,d1112354,0,0.0


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,57715.0,58055.0,58386.0,58726.0,59063.0,59440.0,59840.0,60243.0,60528.0,60657.0,60586.0,60366.0,60103.0,59980.0,60096.0,...,83200.0,85451.0,87277.0,89005.0,90853.0,92898.0,94992.0,97017.0,98737.0,100031.0,100834.0,101222.0,101358.0,101455.0,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,106314.0,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,10174836.0,10399926.0,10637063.0,10893776.0,11173642.0,11475445.0,11791215.0,12108963.0,12412950.0,12689160.0,12943093.0,13171306.0,13341198.0,13411056.0,13356511.0,...,18853437.0,19357126.0,19737765.0,20170844.0,20779953.0,21606988.0,22600770.0,23680871.0,24726684.0,25654277.0,26433049.0,27100536.0,27722276.0,28394813.0,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,5781214.0,5774243.0,5771652.0,5803254.0,5890365.0,6040777.0,6248552.0,6496962.0,6761380.0,7024000.0,7279509.0,7533735.0,7790707.0,8058067.0,8341289.0,...,14400719.0,14871570.0,15359601.0,15866869.0,16395473.0,16945753.0,17519417.0,18121479.0,18758145.0,19433602.0,20149901.0,20905363.0,21695634.0,22514281.0,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,31825295.0,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,2081695.0,2135479.0,2187853.0,2243126.0,2296752.0,2350124.0,2404831.0,2458526.0,2513546.0,2566266.0,2617832.0,2671997.0,...,3168033.0,3148281.0,3128530.0,3108778.0,3089027.0,3060173.0,3051010.0,3039616.0,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,19647.0,20758.0,21890.0,23058.0,24276.0,25559.0,26892.0,28232.0,29520.0,30705.0,31777.0,32771.0,33737.0,34818.0,36067.0,...,64360.0,64327.0,64142.0,64370.0,65390.0,67341.0,70049.0,73182.0,76244.0,78867.0,80993.0,82684.0,83862.0,84463.0,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,77142.0,NaN


### pandas_profile

Read about this [here](https://www.kaggle.com/nulldata/intro-to-pandas-profiling-simple-fast-eda)

In [5]:
# Cardinality характеризует уникальность данных.
# Высокая кардинальность - уникальные данные,
# низкая кардинальность - повторяющиеся данные.
# profile = ProfileReport(food_data, config_file="./input/config_dark.yaml")
# profile.to_file("project3.html")

In [6]:
index = food_data.index

number_of_rows = len(index)
uniq_vs_nan = pd.DataFrame(index=food_data.columns, columns=["uniq", "nulled"])

for col in tqdm(food_data.columns):
    # this is more easy way to countiing nulled values
    uniq_vs_nan.loc[col, "nulled"] = food_data[col].isnull().sum(axis=0)
    # try:
    #    nan_count = pd.notnull(food_data[col]).value_counts()[False]
    # except KeyError:  # нет значений nan
    #    nan_count = 0
    uniq_vs_nan.loc[col, "uniq"] = food_data[col].nunique()


fig = go.Figure(
    data=[
        go.Bar(
            name="Uniq values (abs)",
            x=uniq_vs_nan.index,
            y=uniq_vs_nan["uniq"],
        ),
        go.Bar(
            name="NaN values (abs)",
            x=uniq_vs_nan.index,
            y=uniq_vs_nan["nulled"],
        ),
    ]
)
# Change the bar mode
fig.update_layout(barmode="group")
fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)",},
    title={
        "text": "uniq to NaN",
        "y": 0.85,
        "x": 0.4,
        "xanchor": "center",
        "yanchor": "top",
    },
    xaxis_title="Values count",
    yaxis_title="Column name",
    font={"family": "Courier New, monospace", "size": 14, "color": "#7f7f7f"},
)
fig.show()

In [7]:
# if the zero values are rather low, they may not be visible in the chart
# check all columns that have zero values.
display(uniq_vs_nan[uniq_vs_nan.nulled > 0])

,uniq,nulled
Cuisine Style,10731,11590
Price Range,3,17361
Number of Reviews,1573,3200
Reviews,41857,2


In [8]:
# just other zero values checking method 
# duble check matter :)))
null_value_stats = food_data.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]
display(null_value_stats)

Restaurant_id            0
City                     0
Cuisine Style        11590
Ranking                  0
Price Range          17361
Number of Reviews     3200
Reviews                  2
URL_TA                   0
ID_TA                    0
sample                   0
Rating                   0
dtype: int64

### Build target variable distribution graph

In [9]:
fig = go.Figure(
    data=[
        go.Bar(
            name="Rating",
            x=food_data["Rating"].value_counts().index,
            y=food_data["Rating"].value_counts(),
        )
    ]
)
# Change the bar mode
fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)",},
    title={
        "text": "Target var distribution",
        "y": 0.85,
        "x": 0.4,
        "xanchor": "center",
        "yanchor": "top",
    },
    xaxis_title="Rating value",
    yaxis_title="Rating value count",
    font={"family": "Courier New, monospace", "size": 14, "color": "#7f7f7f"},
)
fig.show()

#### We have 10 000 rating values equal 0 from test dataset

### Saving information about **nan**

In [10]:
# we have nan in 3 columns: "Cuisine Style", "Price Range", "Number of Reviews"
# don't sav info about 'Reviews' because it's only 2 nan values
# save information about nan's in dataset
def save_nan(param):
    return food_data[param].apply(lambda x: 1 if pd.isna(x) else 0)


food_data["NO Cuisine Style"] = save_nan("Cuisine Style")
food_data["NO Price Range"] = save_nan("Price Range")
food_data["NO Number of Reviews"] = save_nan("Number of Reviews")
food_data['Number of Reviews'].fillna(0, inplace=True)

# check
display(food_data[food_data["NO Cuisine Style"] == 1].head(5))

,Restaurant_id,City,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,sample,Rating,NO Cuisine Style,NO Price Range,NO Number of Reviews
7,id_7,Budapest,NaN,2330.0,NaN,0.0,"[[], []]",/Restaurant_Review-g274887-d8286886-Reviews-Be...,d8286886,0,0.0,1,1,1
10,id_10,Rome,NaN,1209.0,NaN,306.0,"[['Very pretentious, but drinks are top notch'...",/Restaurant_Review-g187791-d7818546-Reviews-Je...,d7818546,0,0.0,1,1,0
13,id_13,Hamburg,NaN,1000.0,NaN,30.0,"[['Surprisingly good pizza', 'Yummy'], ['09/28...",/Restaurant_Review-g187331-d1344523-Reviews-Il...,d1344523,0,0.0,1,1,0
18,id_18,Vienna,NaN,3360.0,NaN,2.0,"[[], []]",/Restaurant_Review-g190454-d8435085-Reviews-An...,d8435085,0,0.0,1,1,0
20,id_20,Barcelona,NaN,6085.0,NaN,8.0,"[[], []]",/Restaurant_Review-g187497-d8334752-Reviews-Re...,d8334752,0,0.0,1,1,0


In [11]:
food_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant_id         50000 non-null  object 
 1   City                  50000 non-null  object 
 2   Cuisine Style         38410 non-null  object 
 3   Ranking               50000 non-null  float64
 4   Price Range           32639 non-null  object 
 5   Number of Reviews     50000 non-null  float64
 6   Reviews               49998 non-null  object 
 7   URL_TA                50000 non-null  object 
 8   ID_TA                 50000 non-null  object 
 9   sample                50000 non-null  int64  
 10  Rating                50000 non-null  float64
 11  NO Cuisine Style      50000 non-null  int64  
 12  NO Price Range        50000 non-null  int64  
 13  NO Number of Reviews  50000 non-null  int64  
dtypes: float64(3), int64(4), object(7)
memory usage: 5.3+ MB


# 1. Price Range <a name="price_range"></a>

In [12]:
food_data["Price Range"].value_counts()

$$ - $$$    23041
$            7816
$$$$         1782
Name: Price Range, dtype: int64

In [13]:
# We have 17361 NaN's in "Price Range"
# Most popular value  is "$$ - $$$"
# when fill all nan  with most popular

food_data["Price Range"].fillna(
    food_data["Price Range"].mode()[0], inplace=True
)
# check values
print(food_data["Price Range"].value_counts())

$$ - $$$    40402
$            7816
$$$$         1782
Name: Price Range, dtype: int64


In [14]:
# Convert "Price Range" to numeerical
price_range_dict = {"$": 1.0, "$$ - $$$": 2.0, "$$$$": 3.0}
food_data["Price Range"] = food_data["Price Range"].apply(
    lambda x: price_range_dict[x]
)
# check
print(food_data["Price Range"].value_counts())
print(food_data.info())

2.0    40402
1.0     7816
3.0     1782
Name: Price Range, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant_id         50000 non-null  object 
 1   City                  50000 non-null  object 
 2   Cuisine Style         38410 non-null  object 
 3   Ranking               50000 non-null  float64
 4   Price Range           50000 non-null  float64
 5   Number of Reviews     50000 non-null  float64
 6   Reviews               49998 non-null  object 
 7   URL_TA                50000 non-null  object 
 8   ID_TA                 50000 non-null  object 
 9   sample                50000 non-null  int64  
 10  Rating                50000 non-null  float64
 11  NO Cuisine Style      50000 non-null  int64  
 12  NO Price Range        50000 non-null  int64  
 13  NO Number of Reviews  50000 non-null  int64  
dtyp

# 2. City <a name="city"></a>

In [15]:
food_data["City"].value_counts()

London        7193
Paris         6108
Madrid        3813
Barcelona     3395
Berlin        2705
Milan         2694
Rome          2589
Prague        1816
Lisbon        1647
Vienna        1443
Amsterdam     1372
Brussels      1314
Hamburg       1186
Munich        1145
Lyon          1126
Stockholm     1033
Budapest      1021
Warsaw         925
Dublin         836
Copenhagen     819
Athens         776
Edinburgh      745
Zurich         685
Oporto         642
Geneva         590
Krakow         554
Oslo           488
Helsinki       475
Bratislava     380
Luxembourg     263
Ljubljana      222
Name: City, dtype: int64

### Gather information about cities from **world_cityes** dataset and save it do **food_data** dataset

In [16]:
# use only capitals in dataset
world_cityes = world_cityes.dropna(subset=["capital"])

# change Oporto to Porto - it's same
food_data.loc[food_data["City"] == "Oporto", "City"] = "Porto"


def get_dict_by_city(param):
    return pd.Series(
        world_cityes[param].values, index=world_cityes["city_ascii"]
    ).to_dict()


population_dict = get_dict_by_city("population")
capital_dict = get_dict_by_city("capital")
country_code_dict = get_dict_by_city("iso3")

food_data["population in thousands"] = food_data["City"].apply(
    lambda x: population_dict[x] / 1000
)
food_data["capital"] = food_data["City"].apply(lambda x: capital_dict[x])
food_data["country_code"] = food_data["City"].apply(
    lambda x: country_code_dict[x]
)

display(food_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Restaurant_id            50000 non-null  object 
 1   City                     50000 non-null  object 
 2   Cuisine Style            38410 non-null  object 
 3   Ranking                  50000 non-null  float64
 4   Price Range              50000 non-null  float64
 5   Number of Reviews        50000 non-null  float64
 6   Reviews                  49998 non-null  object 
 7   URL_TA                   50000 non-null  object 
 8   ID_TA                    50000 non-null  object 
 9   sample                   50000 non-null  int64  
 10  Rating                   50000 non-null  float64
 11  NO Cuisine Style         50000 non-null  int64  
 12  NO Price Range           50000 non-null  int64  
 13  NO Number of Reviews     50000 non-null  int64  
 14  population in thousand

None

### Gather information about Countries population from **country_population** dataset and save it do **food_data** dataset

In [17]:
# country_pop_dict = pd.Series(
#     country_population["2018"].values, index=country_population["Country Code"]
# ).to_dict()
# # country_pop_dict = country_population[['Country Code','2018']].to_dict()


# food_data["Population in country"] = food_data["country_code"].apply(lambda x: country_pop_dict[x])


### Build restaraunt per City graph

In [18]:
fig = go.Figure(
    data=[
        go.Bar(
            name="Rest peer City",
            x=food_data["City"].value_counts().index,
            y=food_data["City"].value_counts(),
        ),
        go.Bar(
            name="Population",
            y=food_data["population in thousands"].value_counts().index,
            x=food_data["City"].value_counts().index
        ),
    ]
)
# Change the bar mode
fig.update_layout(barmode="group")
fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)",},
    title={
        "text": "Rest peer City",
        "y": 0.85,
        "x": 0.4,
        "xanchor": "center",
        "yanchor": "top",
    },
    xaxis_title="City name",
    yaxis_title="Rest count",
    font={"family": "Courier New, monospace", "size": 14, "color": "#7f7f7f"},
)
fig.show()

In [19]:
# add new vars: 1. rest count per City, 2. rest count per 1 000 population
rest_counts_dict = food_data["City"].value_counts().to_dict()
food_data["Rest counts per City"] = food_data["City"].apply(
    lambda x: rest_counts_dict[x]
)
food_data["Rest counts per 1000"] = (
    food_data["Rest counts per City"] / food_data["population in thousands"]
)
display(food_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Restaurant_id            50000 non-null  object 
 1   City                     50000 non-null  object 
 2   Cuisine Style            38410 non-null  object 
 3   Ranking                  50000 non-null  float64
 4   Price Range              50000 non-null  float64
 5   Number of Reviews        50000 non-null  float64
 6   Reviews                  49998 non-null  object 
 7   URL_TA                   50000 non-null  object 
 8   ID_TA                    50000 non-null  object 
 9   sample                   50000 non-null  int64  
 10  Rating                   50000 non-null  float64
 11  NO Cuisine Style         50000 non-null  int64  
 12  NO Price Range           50000 non-null  int64  
 13  NO Number of Reviews     50000 non-null  int64  
 14  population in thousand

None

### Build rest per Country graph

In [20]:
fig = go.Figure(
    data=[
        go.Bar(
            name="Rest peer Country",
            x=food_data["country_code"].value_counts().index,
            y=food_data["country_code"].value_counts(),
        )
    ]
)
# Change the bar mode
fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)",},
    title={
        "text": "Rest peer Country",
        "y": 0.85,
        "x": 0.4,
        "xanchor": "center",
        "yanchor": "top",
    },
    xaxis_title="Country code",
    yaxis_title="Rest count",
    font={"family": "Courier New, monospace", "size": 14, "color": "#7f7f7f"},
)
fig.show()

In [21]:
# add new var - rest count per Country
rest_counts_dict = food_data["country_code"].value_counts().to_dict()
food_data["Rest counts per Country"] = food_data["country_code"].apply(
    lambda x: rest_counts_dict[x]
)

In [22]:
display(food_data["Rest counts per Country"].value_counts())

7938    7938
7234    7234
5283    5283
5036    5036
3813    3813
3395    3395
2289    2289
1816    1816
1479    1479
1443    1443
1372    1372
1314    1314
1275    1275
1033    1033
1021    1021
836      836
819      819
776      776
488      488
475      475
380      380
263      263
222      222
Name: Rest counts per Country, dtype: int64

# 3. Cuisine Style

**Cuisine Style** column looks like a python list. For prevent assertion in _ast.literal_eval()_ we should replace nan's with _"['Other']"_ value

In [23]:
# replace NaN to ['Other'] to prevent assertion of ast.literal_eval
food_data["Cuisine Style"].fillna("['Other']", inplace=True)
food_data["Cuisine Style"] = food_data["Cuisine Style"].apply(
    lambda x: ast.literal_eval(x)
)

flat_list = [
    item for sublist in food_data["Cuisine Style"] for item in sublist
]
cuisine_style_counter = Counter(flat_list)

print(len(cuisine_style_counter))
pp.pprint((take(10, cuisine_style_counter.most_common())))

126
[   ('Vegetarian Friendly', 14021),
    ('European', 12561),
    ('Other', 11590),
    ('Mediterranean', 7820),
    ('Italian', 7433),
    ('Vegan Options', 5684),
    ('Gluten Free Options', 5185),
    ('Bar', 4094),
    ('French', 3970),
    ('Asian', 3786)]


In [24]:
# Cuisines to dummyes
for cuisine_style in cuisine_style_counter:
    food_data[cuisine_style] = food_data["Cuisine Style"].apply(
        lambda x: 1 if cuisine_style in x else 0
    )

### Add new columnt to dataset with Cuisine Style count

In [25]:
food_data["Cuisine Style Count"] = food_data["Cuisine Style"].apply(
    lambda x: len(x)
)
# get mean cousine styles per restaurant
pp.pprint(food_data["Cuisine Style Count"].mean())
display(food_data.head(5))

2.62532


,Restaurant_id,City,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,sample,Rating,NO Cuisine Style,NO Price Range,NO Number of Reviews,population in thousands,capital,country_code,Rest counts per City,Rest counts per 1000,Rest counts per Country,Bar,Pub,European,Scandinavian,Gluten Free Options,...,Southwestern,Mongolian,Cajun & Creole,Singaporean,Filipino,Cuban,Native American,Canadian,Egyptian,Azerbaijani,Georgian,Yunnan,New Zealand,Ukrainian,Albanian,Central Asian,Polynesian,Caucasian,Uzbek,Salvadoran,Xinjiang,Burmese,Fujian,Latvian,Cuisine Style Count
0,id_0,Paris,"[Bar, Pub]",12963.0,2.0,4.0,"[[], []]",/Restaurant_Review-g187147-d10746918-Reviews-L...,d10746918,0,0.0,0,0,0,9904.000,primary,FRA,6108,0.616721,7234,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,id_1,Helsinki,"[European, Scandinavian, Gluten Free Options, ...",106.0,2.0,97.0,"[['Very good reviews!', 'Fine dining in Hakani...",/Restaurant_Review-g189934-d6674944-Reviews-Ra...,d6674944,0,0.0,0,0,0,1115.000,primary,FIN,475,0.426009,475,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
2,id_2,Edinburgh,[Vegetarian Friendly],810.0,2.0,28.0,"[['Better than the Links', 'Ivy Black'], ['12/...",/Restaurant_Review-g186525-d13129638-Reviews-B...,d13129638,0,0.0,0,0,0,504.966,admin,GBR,745,1.475347,7938,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,id_3,London,"[Italian, Mediterranean, European, Vegetarian ...",1669.0,3.0,202.0,"[['Most exquisite', 'Delicious and authentic']...",/Restaurant_Review-g186338-d680417-Reviews-Qui...,d680417,0,0.0,0,0,0,8567.000,primary,GBR,7193,0.839617,7938,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
4,id_4,Bratislava,"[Italian, Mediterranean, European, Seafood, Ve...",37.0,3.0,162.0,"[['Always the best in bratislava', 'Very good ...",/Restaurant_Review-g274924-d1112354-Reviews-Ma...,d1112354,0,0.0,0,0,0,423.737,primary,SVK,380,0.896783,380,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6


### Build BoxPlot for Cuisine Style count

In [26]:
fig = px.box(food_data, x="Cuisine Style Count", orientation='h')
fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)",},
    title={
        "text": "BoxPlot по количеству кухонь",
        "y": 0.95,
        "x": 0.4,
        "xanchor": "center",
        "yanchor": "top",
    },
    xaxis_title="Количество кухонь",    
    font={"family": "Courier New, monospace", "size": 14, "color": "#7f7f7f"},
)
fig.show()

# 4. Reviews

Reviews column looks like a python list of lists,but two times we have values like: **"[['Will certainly be back', nan], ['11/26/2017', '07/27/2017']]"** with **nan**. For prevent assertion in _ast.literal_eval()_ we should replace nan's


In [27]:
# food_data["Reviews"] = food_data["Reviews"].str.lower()
food_data["Number of Reviews"].mean()


food_data["Reviews"] = food_data["Reviews"].str.replace("nan", "' '")
# remove 'nan' for prevent catboost fall with assertion
food_data["Reviews"].fillna("[[], []]", inplace=True)

# for debug only(for catch assertion) instead use lambda x: tokenizer.tokenize(x)
# def tokenize_review_debug(tok):
#    try:
#      ret = tokenizer.tokenize(tok)
#    except:
#      print(tok)
#    return ret


# for debug only(for catch assertion) instead use lambda x: ast.literal_eval(x)
# def ast_review_debug(tok):
#   try:
#     ret = ast.literal_eval(tok)
#   except:
#         print(tok)
#     return ret


food_data["Reviews"] = food_data["Reviews"].apply(
    lambda x: ast.literal_eval(x)
)

# how many reviews have the date and text?
# first of all get max and min list of dates len
reviews_dates_list_len = food_data["Reviews"].apply(lambda x: len(x[1]))

print(f"Maximum Reviews with text`s: {reviews_dates_list_len.max()}")
print(f"Minimum Reviews with text`s: {reviews_dates_list_len.min()}")

# ok min is 0 it's mean that we have Reviews with no date
# check how many it is?
reviews_dates_list_0 = food_data["Reviews"].apply(
    lambda x: 1 if len(x[1]) == 0 else 0
)
print(f"Number of rows with no date in Review: {reviews_dates_list_0.sum()}")

# now check how many records have not Review at all
reviews_list_0 = food_data["Reviews"].apply(
    lambda x: 1 if len(x[0]) == 0 else 0
)
print(f"Number of rows with no text in Review:{reviews_dates_list_0.sum()}")

# hmm these numbers are equal, bith is 8114
# now check what we have not rows with dates in Review but without Reviews text
# and save this in dataset

food_data["No Dates or No texts in Reviews"] = food_data["Reviews"].apply(
    lambda x: 1
    if (len(x[0]) == 0 and len(x[1]) != 0)
    or (len(x[1]) == 0 and len(x[0]) != 0)
    else 0
)

print(
    f"Number of rows with text but no date and vice versa: \
    {food_data['No Dates or No texts in Reviews'].sum()}"
)

Maximum Reviews with text`s: 2
Minimum Reviews with text`s: 0
Number of rows with no date in Review: 8114
Number of rows with no text in Review:8114
Number of rows with text but no date and vice versa:     0


### Working with review dates

In [28]:
# convert text to date
food_data["Reviews Dates"] = food_data["Reviews"].apply(
    lambda x: [datetime.strptime(d, "%m/%d/%Y") for d in x[1]]
)

# Check if dates in list in right order and reordering if order is wrong
food_data["Reviews Dates"] = food_data["Reviews Dates"].apply(
    lambda x: [x[i] for i in [1, 0]] if ((len(x) > 1) and (x[1] > x[0])) else x
)

# Add diffs between first and last review to dataset
food_data["Reviews Dates Diff"] = food_data["Reviews Dates"].apply(
    lambda x: (x[0] - x[1]).days if len(x) > 1 else 0
)

# Add number of reviews per restaurant to dataset
food_data["Number of Reviews per rest"] = food_data["Reviews Dates"].apply(
    lambda x: len(x)
)


print(
    f"Maximal number of days between Reviews: \
    {food_data['Reviews Dates Diff'].max()}"
)

print(
    f"Minimal number of days between Reviews: \
    {food_data['Reviews Dates Diff'].min()}"
)


print(
    f"Mean number of days between Reviews: \
    {food_data['Reviews Dates Diff'].mean()}"
)

dates_list = food_data["Reviews Dates"].to_list()
dates_flat = [item for sublist in dates_list for item in sublist]
latest_review = sorted(dates_flat, reverse=True)[0]
earlest_review = sorted(dates_flat)[0]
print(f"Earlest review date: {earlest_review}")
print(f"Latest review date: {latest_review}")

print(
    f'Reviews per restaraunt value counts:\n {food_data["Number of Reviews"].value_counts().to_frame()}'
)

# Add information how far current review is from latest review
food_data["How far from latest review"] = food_data["Reviews Dates"].apply(
    lambda x: (latest_review - x[0]).days
    if len(x) > 0
    else (latest_review - earlest_review).days
)

Maximal number of days between Reviews:     3296
Minimal number of days between Reviews:     0
Mean number of days between Reviews:     103.22246
Earlest review date: 2004-04-21 00:00:00
Latest review date: 2018-02-26 00:00:00
Reviews per restaraunt value counts:
         Number of Reviews
0.0                  3200
2.0                  2403
3.0                  2042
4.0                  1707
5.0                  1471
...                   ...
5216.0                  1
1932.0                  1
1050.0                  1
1145.0                  1
1471.0                  1

[1574 rows x 1 columns]


# 5. Number of Reviews

In [29]:
# add new var - number of Reviews per City
review_counts_dict = (
    food_data.groupby(["City"])["Number of Reviews"].sum().to_dict()
)

food_data["Number of Reviews per City"] = food_data["City"].apply(
    lambda x: review_counts_dict[x]
)

# add new var - number of Reviews per Country
review_counts_dict = (
    food_data.groupby(["country_code"])["Number of Reviews"].sum().to_dict()
)

food_data["Number of Reviews per Country"] = food_data["country_code"].apply(
    lambda x: review_counts_dict[x]
)

In [30]:
food_data.info()
display(food_data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 154 entries, Restaurant_id to Number of Reviews per Country
dtypes: float64(8), int64(137), object(9)
memory usage: 58.7+ MB


,Restaurant_id,City,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,sample,Rating,NO Cuisine Style,NO Price Range,NO Number of Reviews,population in thousands,capital,country_code,Rest counts per City,Rest counts per 1000,Rest counts per Country,Bar,Pub,European,Scandinavian,Gluten Free Options,...,Canadian,Egyptian,Azerbaijani,Georgian,Yunnan,New Zealand,Ukrainian,Albanian,Central Asian,Polynesian,Caucasian,Uzbek,Salvadoran,Xinjiang,Burmese,Fujian,Latvian,Cuisine Style Count,No Dates or No texts in Reviews,Reviews Dates,Reviews Dates Diff,Number of Reviews per rest,How far from latest review,Number of Reviews per City,Number of Reviews per Country
0,id_0,Paris,"[Bar, Pub]",12963.0,2.0,4.0,"[[], []]",/Restaurant_Review-g187147-d10746918-Reviews-L...,d10746918,0,0.0,0,0,0,9904.000,primary,FRA,6108,0.616721,7234,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,[],0,0,5059,680827.0,774779.0
1,id_1,Helsinki,"[European, Scandinavian, Gluten Free Options, ...",106.0,2.0,97.0,"[[Very good reviews!, Fine dining in Hakaniemi...",/Restaurant_Review-g189934-d6674944-Reviews-Ra...,d6674944,0,0.0,0,0,0,1115.000,primary,FIN,475,0.426009,475,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,"[2017-12-05 00:00:00, 2017-10-29 00:00:00]",37,2,83,29511.0,29511.0
2,id_2,Edinburgh,[Vegetarian Friendly],810.0,2.0,28.0,"[[Better than the Links, Ivy Black], [12/28/20...",/Restaurant_Review-g186525-d13129638-Reviews-B...,d13129638,0,0.0,0,0,0,504.966,admin,GBR,745,1.475347,7938,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"[2017-12-28 00:00:00, 2017-12-06 00:00:00]",22,2,60,172103.0,1105873.0
3,id_3,London,"[Italian, Mediterranean, European, Vegetarian ...",1669.0,3.0,202.0,"[[Most exquisite, Delicious and authentic], [1...",/Restaurant_Review-g186338-d680417-Reviews-Qui...,d680417,0,0.0,0,0,0,8567.000,primary,GBR,7193,0.839617,7938,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,"[2017-12-06 00:00:00, 2017-11-22 00:00:00]",14,2,82,933770.0,1105873.0
4,id_4,Bratislava,"[Italian, Mediterranean, European, Seafood, Ve...",37.0,3.0,162.0,"[[Always the best in bratislava, Very good but...",/Restaurant_Review-g274924-d1112354-Reviews-Ma...,d1112354,0,0.0,0,0,0,423.737,primary,SVK,380,0.896783,380,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,"[2018-02-08 00:00:00, 2018-01-24 00:00:00]",15,2,18,16672.0,16672.0
5,id_5,Paris,"[French, Cafe, Fast Food]",5093.0,1.0,24.0,"[[Lunch Break, Good to Quick Bite], [09/07/201...",/Restaurant_Review-g187147-d8458643-Reviews-Le...,d8458643,0,0.0,0,0,0,9904.000,primary,FRA,6108,0.616721,7234,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,"[2017-09-07 00:00:00, 2017-09-02 00:00:00]",5,2,172,680827.0,774779.0
6,id_6,Lisbon,"[European, Portuguese]",3704.0,1.0,2.0,"[[], []]",/Restaurant_Review-g189158-d8758873-Reviews-Li...,d8758873,0,0.0,0,0,0,2812.000,primary,PRT,1647,0.585704,2289,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,[],0,0,5059,196347.0,276702.0
7,id_7,Budapest,[Other],2330.0,2.0,0.0,"[[], []]",/Restaurant_Review-g274887-d8286886-Reviews-Be...,d8286886,0,0.0,1,1,1,1679.000,primary,HUN,1021,0.608100,1021,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,[],0,0,5059,148366.0,148366.0
8,id_8,Helsinki,"[European, Scandinavian]",771.0,2.0,2.0,"[[Big group supper, Good food but very slow se...",/Restaurant_Review-g189934-d12947542-Reviews-B...,d12947542,0,0.0,0,1,0,1115.000,primary,FIN,475,0.426009,475,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,"[2018-01-07 00:00:00, 2017-10-11 00:00:00]",88,2,50,29511.0,29511.0
9,id_9,Stockholm,[Italian],605.0,2.0,13.0,"[[Good food, Good pizza], [12/02/2017, 10/29/2...",/Restaurant_Review-g189852-d12716384-Reviews-I...,d12716384,0,0.0,0,1,0,1264.000,primary,SWE,1033,0.817247,1033,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"[2017-12-02 00:00:00, 2017-10-29 00:00:00]",34,2,86,71588.0,71588.0


## Working with Reviews texts

We will try to work with reviews texts using catboost text tools. For more details see [this](https://github.com/catboost/catboost/blob/564d8c016a04d088f8af75d9bc5d986731188a11/catboost/tutorials/events/2020_06_04_catboost_tutorial/text_features.ipynb)

### Tokenize words

In [31]:
# https://www.kaggle.com/residentmario/exploring-tripadvisor-uk-restaurant-reviews
#

# lets go to work with Review texts

# add  "Number" to token_types for number processing
tokenizer = Tokenizer(
    lowercasing=True, separator_type="BySense", token_types=["Word"]
)


stop_words = set(
    ("be", "is", "are", "the", "an", "of", "and", "in", "food", "a")
)


def filter_stop_words(tokens):
    return list(filter(lambda x: x not in stop_words, tokens))


reviews_tokens = food_data["Reviews"].apply(
    lambda x: [
        item
        for sublist in [tokenizer.tokenize(i) for i in x[0]]
        for item in sublist
    ]
)
reviews_tokens = [filter_stop_words(tokens) for tokens in reviews_tokens]

# convert list of lists to list
flat_list = [item for sublist in reviews_tokens for item in sublist]
tokens_counter = Counter(flat_list)

all_tokens_count = 0

review_meaning_words = [
    "awesome",
    "excellent",
    "great",
    "good",
    "ok",
    "bad",
    "awful",
    "terrible",
    "horrible",
]


for w in review_meaning_words:
    try:
        all_tokens_count += tokens_counter[w]
        print(f'Token {w} occured {tokens_counter[w]} times')
    except KeyError:
        print(f"word {w} not found!")

print(f"Total tokens count: {all_tokens_count}")

Token awesome occured 440 times
Token excellent occured 3257 times
Token great occured 10626 times
Token good occured 11770 times
Token ok occured 762 times
Token bad occured 718 times
Token awful occured 183 times
Token terrible occured 331 times
Token horrible occured 150 times
Total tokens count: 28237


### Lemalize words

In [32]:
# import nltk
# import os

# nltk_data_path = os.path.join(os.path.dirname(nltk.__file__), 'nltk_data')
# nltk.data.path.append(nltk_data_path)
# nltk.download('wordnet', nltk_data_path)

# lemmatizer = nltk.stem.WordNetLemmatizer()

# def lemmatize_tokens_nltk(tokens):
#     return list(map(lambda t: lemmatizer.lemmatize(t), tokens))


# text_small_lemmatized_nltk = [lemmatize_tokens_nltk(tokens) for tokens in reviews_tokens]
# # convert list of lists to list
# flat_list = [item for sublist in text_small_lemmatized_nltk for item in sublist]
# text_small_lemmatized_nltk_counter = Counter(flat_list)

# total_lemmas_count = 0
# for w in [
#     "awesome",
#     "excellent",
#     "great",
#     "good",
#     "ok",
#     "bad",
#     "awful",
#     "terrible",
#     "horrible",
# ]:
#     try:
#         # print(text_small_lemmatized_nltk_counter[w])
#         total_lemmas_count += text_small_lemmatized_nltk_counter[w]
#     except:
#         print(f"word {w} not found!")

# print(f'Total lemmas count: {total_lemmas_count}')
# print(f'Lemmas and tokens count diff: {total_lemmas_count-all_tokens_count}')

We not need lemmalization. Dofference between tokens and lemmas is too small

### Write info about words in dataset

In [33]:
def review_word_counter(rev, word):
    ret = 0
    for i in rev[0]:
        ret += i.count(word)
    return ret


for w in review_meaning_words:
    food_data[w] = food_data["Reviews"].apply(review_word_counter, args=(w,))

In [34]:
display(food_data.head(5))

,Restaurant_id,City,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,sample,Rating,NO Cuisine Style,NO Price Range,NO Number of Reviews,population in thousands,capital,country_code,Rest counts per City,Rest counts per 1000,Rest counts per Country,Bar,Pub,European,Scandinavian,Gluten Free Options,...,Polynesian,Caucasian,Uzbek,Salvadoran,Xinjiang,Burmese,Fujian,Latvian,Cuisine Style Count,No Dates or No texts in Reviews,Reviews Dates,Reviews Dates Diff,Number of Reviews per rest,How far from latest review,Number of Reviews per City,Number of Reviews per Country,awesome,excellent,great,good,ok,bad,awful,terrible,horrible
0,id_0,Paris,"[Bar, Pub]",12963.0,2.0,4.0,"[[], []]",/Restaurant_Review-g187147-d10746918-Reviews-L...,d10746918,0,0.0,0,0,0,9904.000,primary,FRA,6108,0.616721,7234,1,1,0,0,0,...,0,0,0,0,0,0,0,0,2,0,[],0,0,5059,680827.0,774779.0,0,0,0,0,0,0,0,0,0
1,id_1,Helsinki,"[European, Scandinavian, Gluten Free Options, ...",106.0,2.0,97.0,"[[Very good reviews!, Fine dining in Hakaniemi...",/Restaurant_Review-g189934-d6674944-Reviews-Ra...,d6674944,0,0.0,0,0,0,1115.000,primary,FIN,475,0.426009,475,0,0,1,1,1,...,0,0,0,0,0,0,0,0,5,0,"[2017-12-05 00:00:00, 2017-10-29 00:00:00]",37,2,83,29511.0,29511.0,0,0,0,1,0,0,0,0,0
2,id_2,Edinburgh,[Vegetarian Friendly],810.0,2.0,28.0,"[[Better than the Links, Ivy Black], [12/28/20...",/Restaurant_Review-g186525-d13129638-Reviews-B...,d13129638,0,0.0,0,0,0,504.966,admin,GBR,745,1.475347,7938,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,"[2017-12-28 00:00:00, 2017-12-06 00:00:00]",22,2,60,172103.0,1105873.0,0,0,0,0,0,0,0,0,0
3,id_3,London,"[Italian, Mediterranean, European, Vegetarian ...",1669.0,3.0,202.0,"[[Most exquisite, Delicious and authentic], [1...",/Restaurant_Review-g186338-d680417-Reviews-Qui...,d680417,0,0.0,0,0,0,8567.000,primary,GBR,7193,0.839617,7938,0,0,1,0,1,...,0,0,0,0,0,0,0,0,5,0,"[2017-12-06 00:00:00, 2017-11-22 00:00:00]",14,2,82,933770.0,1105873.0,0,0,0,0,0,0,0,0,0
4,id_4,Bratislava,"[Italian, Mediterranean, European, Seafood, Ve...",37.0,3.0,162.0,"[[Always the best in bratislava, Very good but...",/Restaurant_Review-g274924-d1112354-Reviews-Ma...,d1112354,0,0.0,0,0,0,423.737,primary,SVK,380,0.896783,380,0,0,1,0,1,...,0,0,0,0,0,0,0,0,6,0,"[2018-02-08 00:00:00, 2018-01-24 00:00:00]",15,2,18,16672.0,16672.0,0,0,0,1,0,0,0,0,0


# 6. ID_TA

In [35]:
# remove first 'd' character from ID_TA parameter and convert it to digit
food_data["ID_TA"] = food_data["ID_TA"].apply(lambda x: int(x[1:]))

# 7. URL_TA

In [36]:
# read group ID from URL

food_data["Group ID from URL_TA"] = food_data["URL_TA"].str.extract(
    pat="(-g\d+)"
)
food_data["Group ID from URL_TA"] = food_data["Group ID from URL_TA"].apply(
    lambda x: x[1:]
)
display(food_data["Group ID from URL_TA"].value_counts())

g186338     7193
g187147     6108
g187514     3813
g187497     3395
g187323     2705
            ... 
g950782        2
g3284927       1
g1080936       1
g1439609       1
g6919449       1
Name: Group ID from URL_TA, Length: 74, dtype: int64

In [37]:
url_ta_group_counter = Counter(food_data["Group ID from URL_TA"].to_list())

print(len(url_ta_group_counter))
pp.pprint((take(10, url_ta_group_counter.most_common())))
# looks like it's some geografic region, may be it's city suburb

74
[   ('g186338', 7193),
    ('g187147', 6108),
    ('g187514', 3813),
    ('g187497', 3395),
    ('g187323', 2705),
    ('g187849', 2694),
    ('g187791', 2589),
    ('g274707', 1816),
    ('g189158', 1647),
    ('g190454', 1443)]


In [38]:
# rest counts per "Group ID from URL_TA"

# rest_counts_dict = food_data["Group ID from URL_TA"].value_counts().to_dict()
# food_data["Rest counts peer Group ID from URL_TA"] = food_data[
#     "Group ID from URL_TA"
# ].apply(lambda x: rest_counts_dict[x])

In [39]:
# add new var - number of Reviews per Group ID from URL_TA
# review_counts_dict = (
#     food_data.groupby(["Group ID from URL_TA"])["Number of Reviews"].sum().to_dict()
# )

# food_data["Number of Reviews per Group ID from URL_TA"] = food_data["Group ID from URL_TA"].apply(
#     lambda x: review_counts_dict[x]
# )


In [40]:
# URL_TA grpups to dummyes
for url_ta_group in url_ta_group_counter:
    food_data[url_ta_group] = food_data["Group ID from URL_TA"].apply(
        lambda x: 1 if url_ta_group in x else 0
    )

# 8. Ranking

### Lets build BoxPlot for Ranking

In [41]:
fig = px.box(food_data, x="Ranking", orientation='h')
fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)",},
    title={
        "text": "Ranking BoxPlot",
        "y": 0.95,
        "x": 0.4,
        "xanchor": "center",
        "yanchor": "top",
    },
    xaxis_title="Rank",    
    font={"family": "Courier New, monospace", "size": 14, "color": "#7f7f7f"},
)
fig.show()

### Now check Ranking distribution in the most common Cities

In [42]:
fig = px.histogram(
    food_data[
        food_data["City"].isin(
            ["London", "Paris", "Madrid", "Barcelona", "Berlin"]
        )
    ],
    x="Ranking",
    nbins=200,
    color="City",
)

fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)",},
    title={
        "text": "Ranking Distribution",
        "y": 0.92,
        "x": 0.4,
        "xanchor": "center",
        "yanchor": "top",
    },
    xaxis_title="Rank",
    yaxis_title="Rest count",
    font={"family": "Courier New, monospace", "size": 14, "color": "#7f7f7f"},
)

fig.show()

### Now we check the Ranking distribution in the least common cities.

In [43]:
fig = px.histogram(
    food_data[
        food_data["City"].isin(
            ["Heelsinki", "Bratislava", "Luxembourg", "Ljubljana", "Oslo"]
        )
    ],
    x="Ranking",
    nbins=200,
    color="City",
)

fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)",},
    title={
        "text": "Ranking Distribution",
        "y": 0.92,
        "x": 0.4,
        "xanchor": "center",
        "yanchor": "top",
    },
    xaxis_title="Rank",
    yaxis_title="Rest count",
    font={"family": "Courier New, monospace", "size": 14, "color": "#7f7f7f"},
)

fig.show()

In [44]:
# add new var - Ranking per Country
review_counts_dict = (
    food_data.groupby(["country_code"])["Ranking"].sum().to_dict()
)

food_data["Ranking per Country"] = food_data["country_code"].apply(
    lambda x: review_counts_dict[x]
)


food_data["Ranking per Country"] = (
    food_data["Ranking per Country"] / food_data["Rest counts per Country"]
)

In [45]:
# Create four new params

food_data["Relative Ranking per Rest counts (City)"] = (
    food_data["Ranking"] / food_data["Rest counts per City"]
)

food_data["Relative Ranking per Number of Reviews (City)"] = (
    food_data["Ranking"] / food_data["Number of Reviews per City"]
)

food_data["Relative Ranking per Rest counts (Country)"] = (
    food_data["Ranking per Country"] / food_data["Rest counts per Country"]
)

food_data["Relative Ranking per Number of Reviews (Country)"] = (
    food_data["Ranking per Country"] / food_data["Number of Reviews per Country"]
)



#### As we can see, the distributions are similar.

# 9. Train model

## Create dummy vars

In [46]:
food_data = pd.get_dummies(
    food_data, columns=["country_code", "capital"]
)

## Clear  from non digital data

In [47]:
food_data = food_data.select_dtypes(exclude=['object'])
food_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 259 entries, Ranking to capital_primary
dtypes: float64(13), int64(221), uint8(25)
memory usage: 90.5 MB


## Split model to test and train 

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics


food_data_train = food_data[food_data["sample"] == 1].drop(["sample"], axis=1)
food_data_test = food_data[food_data["sample"] == 0].drop(["sample"], axis=1)

y = food_data_train["Rating"].values
X = food_data_train.drop(["Rating"], axis=1)


### Train the model

In [59]:
RANDOM_SEED = 42

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED
)
model = RandomForestRegressor(
    n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = np.round(y_pred * 2) / 2

print("MAE:", metrics.mean_absolute_error(y_test, y_pred))

feat_importances = pd.Series(model.feature_importances_, index=X.columns)
display(feat_importances.nlargest(15))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   28.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
MAE: 0.16175


Relative Ranking per Rest counts (City)             0.565341
Number of Reviews                                   0.147125
Relative Ranking per Number of Reviews (City)       0.087904
ID_TA                                               0.026693
Ranking                                             0.025480
How far from latest review                          0.016487
Number of Reviews per City                          0.013702
Reviews Dates Diff                                  0.012263
g187791                                             0.010483
Cuisine Style Count                                 0.005628
Relative Ranking per Number of Reviews (Country)    0.004326
Rest counts per 1000                                0.004193
population in thousands                             0.003809
NO Number of Reviews                                0.002890
Relative Ranking per Rest counts (Country)          0.002650
dtype: float64

In [60]:
test_data = food_data_test.drop(["Rating"], axis=1)

In [61]:
sample_submission = pd.read_csv('./input/sample_submission.csv')

In [52]:
test_data

,Ranking,Price Range,Number of Reviews,ID_TA,NO Cuisine Style,NO Price Range,NO Number of Reviews,population in thousands,Rest counts per City,Rest counts per 1000,Rest counts per Country,Bar,Pub,European,Scandinavian,Gluten Free Options,Vegetarian Friendly,Vegan Options,Italian,Mediterranean,Seafood,French,Cafe,Fast Food,Portuguese,...,country_code_AUT,country_code_BEL,country_code_CHE,country_code_CZE,country_code_DEU,country_code_DNK,country_code_ESP,country_code_FIN,country_code_FRA,country_code_GBR,country_code_GRC,country_code_HUN,country_code_IRL,country_code_ITA,country_code_LUX,country_code_NLD,country_code_NOR,country_code_POL,country_code_PRT,country_code_SVK,country_code_SVN,country_code_SWE,country_code_VEN,capital_admin,capital_primary
0,12963.0,2.0,4.0,10746918,0,0,0,9904.000,6108,0.616721,7234,1,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,106.0,2.0,97.0,6674944,0,0,0,1115.000,475,0.426009,475,0,0,1,1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,810.0,2.0,28.0,13129638,0,0,0,504.966,745,1.475347,7938,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1669.0,3.0,202.0,680417,0,0,0,8567.000,7193,0.839617,7938,0,0,1,0,1,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,37.0,3.0,162.0,1112354,0,0,0,423.737,380,0.896783,380,0,0,1,0,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4367.0,1.0,28.0,11916827,0,0,0,8567.000,7193,0.839617,7938,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9996,2612.0,2.0,2.0,12701633,0,0,0,2812.000,1647,0.585704,2289,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
9997,5562.0,2.0,2.0,13198194,1,1,0,2945.000,2694,0.914771,5283,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
9998,4010.0,2.0,2.0,3862815,0,1,0,2945.000,2694,0.914771,5283,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


In [62]:
predict_submission = model.predict(test_data)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


In [63]:
predict_submission = np.round(predict_submission*2)/2
predict_submission.shape

(10000,)

In [64]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('./input/submission.csv', index=False)
sample_submission.head(10)

,Restaurant_id,Rating
0,id_0,3.0
1,id_1,4.0
2,id_2,4.5
3,id_3,4.5
4,id_4,4.5
5,id_5,4.5
6,id_6,2.5
7,id_7,2.5
8,id_8,4.0
9,id_9,4.5
